In [78]:
"""
First attempt at Lasagne with my real data.
"""

from __future__ import print_function

import sys
import os
import time
import chb

import numpy as np
import theano
import theano.tensor as T

import lasagne
import lasagne.objectives as lobj
import lasagne.nonlinearities as lnon
import lasagne.layers as llay
import lasagne.init as linit


In [79]:
# ################## Download and prepare the CHBMIT dataset ##################
# Loads data for a certain subject (taken as a string 'chbXX').

def load_dataset(subjname):
    # TODO: organize this differently - this should just load the subject and
    #       data, so that we can loop through all the LOOs in main
    # Load data for subject
    subject = chb.CHBsubj()
    subject.load_meta(subjname)
    subject.load_data(exthd=False)

    # Load and read training and test set images and labels.
    x_train, y_train, x_test, y_test = subject.leaveOneOut(1)

    # We reserve the last 100 training examples for validation.
    x_train, x_val = x_train[:-100], x_train[-100:]
    y_train, y_val = y_train[:-100], y_train[-100:]

    # We just return all the arrays in order, as expected in main().
    # (It doesn't matter how we do this as long as we can read them again.)
    return x_train, y_train, x_val, y_val, x_test, y_test

In [65]:
# ##################### Build the neural network model #######################
def build_cnn(input_var=None):
    # The most simple, straightforward CNN I can come up with.
    
    # Input layer, as usual:
    l_in = llay.InputLayer(shape=(None, 1, 23, 256),
                                        input_var=input_var)

    # Convolutional layer with 32 kernels of size 1x32. Strided and padded
    # convolutions are supported as well; see the docstring.
    l_hid1 = llay.Conv2DLayer(l_in, num_filters=16, filter_size=(1, 32),
                                        stride=(1,8), nonlinearity=lnon.sigmoid,
                                        W=linit.GlorotUniform())

    # Another convolution with 32 5x5 kernels, and another 2x2 pooling:
    '''network = llay.Conv2DLayer(
            network, num_filters=32, filter_size=(1, 16),
            nonlinearity=lasagne.nonlinearities.rectify)
    network = llay.MaxPool2DLayer(network, pool_size=(2, 2))
    '''
    # A fully-connected layer of 256 units with 25% dropout on its inputs:
    l_hid2 = llay.DenseLayer(
            llay.dropout(l_hid1, p=.25),
            num_units=256,
            nonlinearity=lnon.sigmoid)

    # And, finally, the 2-unit output layer with 25% dropout on its inputs:
    out = llay.DenseLayer(
            llay.dropout(l_hid2, p=.25),
            num_units=2,
            nonlinearity=lnon.sigmoid)

    return out

In [57]:
# ############################# Batch iterator ###############################
# This is just a simple helper function iterating over training data in
# mini-batches of a particular size, optionally in random order. It assumes
# data is available as numpy arrays. For big datasets, you could load numpy
# arrays as memory-mapped files (np.load(..., mmap_mode='r')), or write your
# own custom data iteration function. For small datasets, you can also copy
# them to GPU at once for slightly improved performance. This would involve
# several changes in the main program, though, and is not demonstrated here.
# Notice that this function returns only mini-batches of size `batchsize`.
# If the size of the data is not a multiple of `batchsize`, it will not
# return the last (remaining) mini-batch.

def iterate_minibatches(inputs, targets, batchsize, shuffle=False):
    assert len(inputs) == len(targets)
    if shuffle:
        indices = np.arange(len(inputs))
        np.random.shuffle(indices)
    for start_idx in range(0, len(inputs) - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield inputs[excerpt], targets[excerpt]

In [7]:
subject='chb01'
# Load the dataset
x_train, y_train, x_val, y_val, x_test, y_test = load_dataset(subject)

Loading: /Users/adamcellon/Drive/senior/thesis/data/chb01.npz
Done: 76.592338 seconds elapsed.


In [58]:
# Prepare Theano variables for inputs and targets
input_var = T.tensor4('inputs')
target_var = T.ivector('targets')

In [66]:
# Create neural network model (depending on first command line parameter)
print("Building model and compiling functions...")
network = build_cnn(input_var)

Building model and compiling functions...


In [68]:
# Create a loss expression for training, i.e., a scalar objective we want
# to minimize (for our 2-class problem, it is the cross-entropy loss):
prediction = llay.get_output(network)

loss = lobj.binary_crossentropy(prediction, target_var)
loss = lobj.aggregate(loss, mode='mean')

In [69]:
# Create update expressions for training, i.e., how to modify the
# parameters at each training step. Here, we'll use Stochastic Gradient
# Descent (SGD).

params = llay.get_all_params(network, trainable=True)
updates = lasagne.updates.sgd(loss, params, learning_rate=0.01)

In [70]:
# Create a loss expression for validation/testing. The crucial difference
# here is that we do a deterministic forward pass through the network,
# disabling dropout layers.

test_prediction = llay.get_output(network, deterministic=True)
test_loss = lobj.binary_crossentropy(test_prediction,target_var)
test_loss = lobj.aggregate(test_loss, mode='mean')

# As a bonus, also create an expression for the classification accuracy:
test_acc = T.mean(lobj.binary_accuracy(test_prediction, target_var),
                  dtype=theano.config.floatX)

In [71]:
# Compile a function performing a training step on a mini-batch (by giving
# the updates dictionary) and returning the corresponding training loss:
train_fn = theano.function([input_var, target_var], loss, updates=updates)

# Compile a second function computing the validation loss and accuracy:
val_fn = theano.function([input_var, target_var], [test_loss, test_acc])

In [72]:
num_epochs=50

# Finally, launch the training loop.
print("Starting training...")
# We iterate over epochs:
for epoch in range(num_epochs):
    # In each epoch, we do a full pass over the training data:
    train_err = 0
    train_batches = 0
    start_time = time.time()
    for batch in iterate_minibatches(x_train, y_train, 2, shuffle=True):
        inputs, targets = batch
        train_err += train_fn(inputs, targets)
        train_batches += 1

    # And a full pass over the validation data:
    val_err = 0
    val_acc = 0
    val_batches = 0
    for batch in iterate_minibatches(x_val, y_val, 2, shuffle=False):
        inputs, targets = batch
        err, acc = val_fn(inputs, targets)
        val_err += err
        val_acc += acc
        val_batches += 1

    # Then we print the results for this epoch:
    print("Epoch {} of {} took {:.3f}s".format(
        epoch + 1, num_epochs, time.time() - start_time))
    print("  training loss:\t\t{:.6f}".format(train_err / train_batches))
    print("  validation loss:\t\t{:.6f}".format(val_err / val_batches))
    print("  validation accuracy:\t\t{:.2f} %".format(
        val_acc / val_batches * 100))

Starting training...
Epoch 1 of 50 took 2.202s
  training loss:		0.746386
  validation loss:		0.671676
  validation accuracy:		61.00 %
Epoch 2 of 50 took 2.117s
  training loss:		0.702222
  validation loss:		0.651293
  validation accuracy:		60.00 %
Epoch 3 of 50 took 2.198s
  training loss:		0.673848
  validation loss:		0.641206
  validation accuracy:		63.50 %
Epoch 4 of 50 took 2.174s
  training loss:		0.653969
  validation loss:		0.653770
  validation accuracy:		61.00 %
Epoch 5 of 50 took 2.146s
  training loss:		0.642635
  validation loss:		0.627299
  validation accuracy:		70.00 %
Epoch 6 of 50 took 2.208s
  training loss:		0.632754
  validation loss:		0.654959
  validation accuracy:		64.00 %
Epoch 7 of 50 took 2.213s
  training loss:		0.622264
  validation loss:		0.634043
  validation accuracy:		66.50 %
Epoch 8 of 50 took 2.018s
  training loss:		0.620116
  validation loss:		0.613205
  validation accuracy:		66.50 %
Epoch 9 of 50 took 2.150s
  training loss:		0.610566
  validation l

In [77]:
# After training, we compute and print the test error:
test_err = 0
test_acc = 0
test_batches = 0
for batch in iterate_minibatches(x_test, y_test, 2, shuffle=False):
    inputs, targets = batch
    err, acc = val_fn(inputs, targets)
    test_err += err
    test_acc += acc
    test_batches += 1
print("Final results:")
print("  test loss:\t\t\t{:.6f}".format(test_err / test_batches))
print("  test accuracy:\t\t{:.2f} %".format(
    test_acc / test_batches * 100))

Final results:
  test loss:			0.696726
  test accuracy:		59.00 %
